## GNN model 

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import sys
import numpy as np
import itertools
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime
import json

In [2]:
cwd = os.getcwd()
print(cwd)
cwd_parent = os.path.abspath(os.path.join(cwd, os.pardir))
print(cwd_parent)

sys.path.append(cwd_parent)

/Users/aliciapliego/Projects/drug_synergy/AI-predict/code
/Users/aliciapliego/Projects/drug_synergy/AI-predict


In [3]:
#Others 
import pickle
import deepadr
from deepadr.dataset import *

In [4]:
import torch
import torch.nn as nn
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset

In [5]:
cwd = os.getcwd()
print(cwd)
cwd_parent = os.path.abspath(os.path.join(cwd, os.pardir))
print(cwd_parent)

sys.path.append(cwd_parent)

/Users/aliciapliego/Projects/drug_synergy/AI-predict/code
/Users/aliciapliego/Projects/drug_synergy/AI-predict


In [6]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [7]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'
targetdata_dir_raw = '../data/processed/dataset_generation/raw'

In [8]:
n_gpu = torch.cuda.device_count()
n_gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
#targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
#targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
#targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = "../data/processed/dataset_generation/experiments"
# # ReaderWriter.dump_data(dpartitions, os.path.join(targetdata_dir, 'data_partitions.pkl'))
print(targetdata_dir_exp)

../data/processed/dataset_generation/experiments


In [10]:
import glob
exp_dirs = glob.glob(targetdata_dir_exp+"/hyp/exp_*")
len(exp_dirs)

0

In [11]:
targetdata_dir_raw = os.path.abspath( "/Users/aliciapliego/Projects/drug_synergy/AI-predict/data/processed/dataset_generation/raw")
#targetdata_dir_processed = os.path.abspath("/Users/aliciapliego/Projects/drug_synergy/AI-predict/data/processed/dataset_generation/processed")

#### Generate Dataset 

In [12]:
targetdata_dir = "/Users/aliciapliego/Projects/drug_synergy/AI-predict/data/processed/dataset_generation/"

In [13]:
targetdata_dir

'/Users/aliciapliego/Projects/drug_synergy/AI-predict/data/processed/dataset_generation/'

In [14]:
targetdata_dir_processed = os.path.abspath("/Users/aliciapliego/Projects/drug_synergy/graphnn_kyriakos/data/processed/DrugComb_zip_thresh_1/data_v1/processed")

In [15]:
dataset = MoleculeDataset(root=targetdata_dir,  dataset='tdcSynergy')

In [16]:
#dataset = MoleculeDataset(root=targetdata_dir,  dataset='tdcSynergy')

In [17]:
dataset.data

PairData(edge_index_a=[2, 15305430], x_a=[7069479, 9], edge_attr_a=[15305430, 3], edge_index_b=[2, 13680958], x_b=[6405928, 9], edge_attr_b=[13680958, 3], id=[233326], y=[233326, 4], expression=[233326, 100])

In [18]:
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}') # number of synergy scores

Dataset: MoleculeDataset(233326):
Number of graphs: 233326
Number of features: 9
Number of classes: 4


In [19]:
# used_dataset = dataset
# If you want to use a smaller subset of the dataset for testing

smaller_dataset_len = int(len(dataset)/5)
used_dataset = dataset[:smaller_dataset_len]

### Hyperparameters

In [20]:
#Hyperparameters
tp = {
   # "batch_size" : 300,
    "num_epochs" : 5,
    
    "emb_dim" : 300,
    "gnn_type" : "gatv2",
    "num_layer" : 5,
    "graph_pooling" : "mean", #attention
    
    "input_embed_dim" : None,
    "gene_embed_dim": 1,
    "num_attn_heads" : 2,
    "num_transformer_units" : 1,
    "p_dropout" : 0.3,
#     "nonlin_func" : nn.ReLU(),
    "mlp_embed_factor" : 2,
    "pooling_mode" : 'attn',
    "dist_opt" : 'cosine',

    "base_lr" : 3e-4, #3e-4
    "max_lr_mul": 5,
    "l2_reg" : 1e-5,
    "loss_w" : 1.,
    "margin_v" : 1.,

    "expression_dim" : 64,
    "expression_input_size" : 908,
    "exp_H1" : 4096,
    "exp_H2" : 2048
}

In [21]:
# input_embed_dim = [128]
# num_attn_heads = [2] # 2,4
# num_transformer_units = [1]
emb_dim = [100]
num_layer = [5]
p_dropout = [0.3]
# nonlin_func = [nn.ReLU()]
# mlp_embed_factor = [2]
# pooling_mode = ['attn']
# dist_opt = ['cosine']
l2_reg = [1e-7] #0
# batch_size = [300]
# num_epochs = [200]
# loss_w = [0.95] # 0.05, 
base_lr = [3e-4]
max_lr_mul = [10]
exp_H1 = [512, 1024, 2048, 4096]
exp_H2 = [512, 1024]


In [22]:
hp_names = ["num_layer",
            "p_dropout",
            "l2_reg",
            "emb_dim",
#             "loss_w",
#            "num_transformer_units",
#            "batch_size",
           "base_lr",
           "max_lr_mul",
           "exp_H1",
           "exp_H2"]

In [23]:
[globals()[i] for i in hp_names]

[[5],
 [0.3],
 [1e-07],
 [100],
 [0.0003],
 [10],
 [512, 1024, 2048, 4096],
 [512, 1024]]

In [24]:
hyperparam_space = list(itertools.product(*[globals()[i] for i in hp_names]))
print(len(hyperparam_space))

8


In [25]:
hyperparam_space[0]

(5, 0.3, 1e-07, 100, 0.0003, 10, 512, 512)

In [26]:
data

<module 'torch.utils.data' from '/usr/local/lib/python3.9/site-packages/torch/utils/data/__init__.py'>

### write hyperparameters into file

In [27]:
num_classes = 4

In [28]:
#targetdata_dir_raw = os.path.abspath( "/Users/aliciapliego/Projects/drug_synergy/AI-predict/data/processed/dataset_generation/raw")
#targetdata_dir_processed = os.path.abspath("/Users/aliciapliego/Projects/drug_synergy/AI-predict/data/processed/dataset_generation/processed")
#targetdata_dir_exp = "/Users/aliciapliego/Projects/drug_synergy/AI-predict/data/processed/dataset_generation/experiments"

exp_dir = os.path.join(targetdata_dir_exp, "exp_"+"try")
state_dict_dir = os.path.join(exp_dir, 'modelstates')


In [29]:

#device_gpu = get_device(True, index=gpu_num)
#print("gpu:", device_gpu)

# Serialize data into file:
json.dump( tp, open( exp_dir + "/hyperparameters.json", 'w' ) )
tp['nonlin_func'] = nn.ReLU()


In [30]:
tp

{'num_epochs': 5,
 'emb_dim': 300,
 'gnn_type': 'gatv2',
 'num_layer': 5,
 'graph_pooling': 'mean',
 'input_embed_dim': None,
 'gene_embed_dim': 1,
 'num_attn_heads': 2,
 'num_transformer_units': 1,
 'p_dropout': 0.3,
 'mlp_embed_factor': 2,
 'pooling_mode': 'attn',
 'dist_opt': 'cosine',
 'base_lr': 0.0003,
 'max_lr_mul': 5,
 'l2_reg': 1e-05,
 'loss_w': 1.0,
 'margin_v': 1.0,
 'expression_dim': 64,
 'expression_input_size': 908,
 'exp_H1': 4096,
 'exp_H2': 2048,
 'nonlin_func': ReLU()}

#### Data partitioning

In [31]:
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit
train_val = ShuffleSplit(n_splits = 2)

In [32]:
for train_index, val_index in train_val.split(dataset):
    print(train_index)
    

[   786 126183 115522 ... 122414  94566 209072]
[217081   4640 222023 ... 223114 135108 122499]


In [33]:
for train_index, test_index in train_val.split(dataset):
    print(train_index)
    

[138409  61772 156304 ... 170676  90384  92697]
[  2382 190731  76957 ... 157722  76911    693]


In [34]:
train_sampler = Subset(dataset, train_index)
validation_sampler = Subset(dataset, val_index)

In [35]:
len(validation_sampler)

23333

In [36]:
len(train_sampler)

209993

In [37]:
batch_size = 16
train_loader = DataLoader(train_sampler, batch_size=batch_size, shuffle=True, follow_batch=['x_a', 'x_b'])
valid_loader = DataLoader(validation_sampler, batch_size=batch_size, shuffle=False, follow_batch=['x_a', 'x_b'])

In [38]:
len(train_loader)

13125

In [39]:
len(valid_loader)

1459

In [40]:
loaders = {"train": train_loader, "valid": valid_loader}


### Graph NN

In [41]:
from deepadr.model_gnn_ogb import GNN, DeepSynergy
gnn_model = GNN(gnn_type = tp["gnn_type"], 
                num_layer = tp["num_layer"], 
                emb_dim = tp["emb_dim"], 
                drop_ratio = 0.5, 
                JK = "multilayer", #last
                graph_pooling = tp["graph_pooling"],
                virtual_node = False,
                with_edge_attr=False)



### Gene expression data model 

In [42]:
#Expression data is StandardScaled
expression_scaler = TorchStandardScaler()
expression_scaler.fit(used_dataset.data.expression)

In [43]:
expression_model = DeepSynergy(D_in=(2*tp["emb_dim"])+tp["expression_input_size"],
                                   H1=tp['exp_H1'], H2=tp['exp_H2'], drop=tp['p_dropout'])
model_name = "ogb"
models = [(gnn_model, f'{model_name}_GNN'),
              (expression_model, f'{model_name}_Expression')]


Dropout(p=0.3, inplace=False) Dropout(p=0.3, inplace=False)


### Training

In [44]:
import torch.nn.functional as F
from torch import nn, optim

In [45]:
#print("Triple shape:",triplet.shape)
print("h_a shape:",h_a.shape)
print("h_b shape:",h_b.shape)
print("h_e shape:",h_e.shape)

NameError: name 'h_a' is not defined

In [46]:
#To match to 700-->
D_in = (2*tp["emb_dim"])+tp["expression_input_size"]

In [47]:
#Check train loader and valid loader contain different data
batch_train = next(iter(train_loader))
batch_valid = next(iter(valid_loader))

print(batch_train.x_a)
print(batch_valid.x_a)

print(batch_train.edge_index_a)
print(batch_valid.edge_index_a)

print(batch_train.edge_attr_a)
print(batch_valid.edge_attr_a)

print(batch_train.x_b)
print(batch_valid.x_b)

print(batch_train.edge_index_b)
print(batch_valid.edge_index_b)

print(batch_train.edge_attr_b)
print(batch_valid.edge_attr_b)

print(batch_train.expression)
print(batch_valid.expression)

print(batch_train.y)
print(batch_valid.y)

tensor([[5, 0, 4,  ..., 2, 0, 1],
        [5, 0, 4,  ..., 2, 0, 1],
        [5, 0, 3,  ..., 1, 0, 1],
        ...,
        [5, 0, 4,  ..., 2, 0, 0],
        [5, 0, 4,  ..., 2, 0, 0],
        [5, 0, 4,  ..., 2, 0, 0]], dtype=torch.int32)
tensor([[5, 0, 4,  ..., 2, 0, 1],
        [5, 0, 4,  ..., 2, 0, 1],
        [5, 0, 4,  ..., 2, 0, 1],
        ...,
        [5, 0, 3,  ..., 1, 0, 1],
        [5, 0, 4,  ..., 2, 0, 1],
        [5, 0, 4,  ..., 2, 0, 0]], dtype=torch.int32)
tensor([[  0,   1,   1,  ..., 426, 450, 434],
        [  1,   0,   2,  ..., 453, 434, 450]], dtype=torch.int32)
tensor([[  0,   1,   1,  ..., 484, 500, 494],
        [  1,   0,   2,  ..., 500, 494, 500]], dtype=torch.int32)
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 1],
        [0, 0, 0],
        [0, 0, 0]], dtype=torch.int32)
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]], dtype=torch.int32)
tensor([

In [52]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

expression_model = DeepSynergy(D_in=700,
                                   H1=500, H2=400,D_out = 4, drop=0.4).to(device=device)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(expression_model.parameters(), lr=0.00001)#old lr = 0.03

train_losses, val_losses = [], []

for epoch in range(tp["num_epochs"]):
    print("=====Epoch {}".format(epoch))
    print('Training...')
    
    
    for m, m_name in models:
        m.train()

    for i_batch, batch in enumerate(tqdm(train_loader)):
        #batch = batch.to(device_gpu)
        
        optimizer.zero_grad()
        
        h_a = gnn_model(batch.x_a, batch.edge_index_a, batch.edge_attr_a, batch.x_a_batch)
        h_b = gnn_model(batch.x_b, batch.edge_index_b, batch.edge_attr_b, batch.x_b_batch)

        h_e = expression_scaler.transform_ondevice(batch.expression, device=device) 
        
        #concatenate Za (drug a), Zb (drug b) and the expression vector scaled
        triplet = torch.cat([h_a, h_b, h_e], axis=-1)

        output = expression_model(triplet)
        #print(output)
        #print(batch.y.type(torch.float))
        
        mean = torch.mean(batch.y, dim=0)
        std = torch.std(batch.y, dim=0)
        normalized_y = (batch.y - mean) / std
        #print(normalized_y)
        
        loss = criterion(output, normalized_y.type(torch.float))
        #print(loss)
        loss.backward() #Derives gradients
        optimizer.step() #Update parameters based on gradients 
        optimizer.zero_grad()  # Clear gradients.
        
        
        train_losses.append(loss.item())
        
    
    
    print(f'epoch {epoch + 1}, loss {loss}')
    
    
    with torch.no_grad():
        expression_model.eval()
        val_losses = []
        r2_scores = []
        mae_scores = []
        rmse_scores = []

        for i_batch, batch in enumerate(tqdm(valid_loader)):
            h_a = gnn_model(batch.x_a, batch.edge_index_a, batch.edge_attr_a, batch.x_a_batch)
            h_b = gnn_model(batch.x_b, batch.edge_index_b, batch.edge_attr_b, batch.x_b_batch)
            h_e = expression_scaler.transform_ondevice(batch.expression, device=device)
            triplet = torch.cat([h_a, h_b, h_e], axis=-1)
            output = expression_model(triplet)

            mean = torch.mean(batch.y, dim=0)
            std = torch.std(batch.y, dim=0)
            normalized_y = (batch.y - mean) / std
            
            loss = criterion(output, normalized_y.type(torch.float))
            val_losses.append(loss.item())

            normalized_y_pred = output.detach().cpu().numpy() * std.numpy() + mean.numpy()
            normalized_y_true = normalized_y.cpu().numpy() * std.numpy()

            r2 = r2_score(normalized_y_true, normalized_y_pred)
            mae = mean_absolute_error(normalized_y_true, normalized_y_pred)
            rmse = mean_squared_error(normalized_y_true, normalized_y_pred, squared=False)

            r2_scores.append(r2)
            mae_scores.append(mae)
            rmse_scores.append(rmse)

            print(f'batch {i_batch}, R-squared: {r2}, MAE: {mae}, RMSE: {rmse}')
            
        avg_val_loss = np.mean(val_losses)
        avg_r2 = np.mean(r2_scores)
        avg_mae = np.mean(mae_scores)
        avg_rmse = np.mean(rmse_scores)
        print(f'validation loss: {avg_val_loss}, R-squared: {avg_r2}, MAE: {avg_mae}, RMSE: {avg_rmse}')



Dropout(p=0.4, inplace=False) Dropout(p=0.4, inplace=False)
=====Epoch 0
Training...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13125/13125 [43:43<00:00,  5.00it/s]


epoch 1, loss 2.8502371311187744


  0%|                                                                                                                                   | 1/1459 [00:00<02:26,  9.97it/s]

batch 0, R-squared: -2.203725694883438, MAE: 20.971290588378906, RMSE: 24.055551528930664


...

batch 1453, R-squared: -2.592068973081395, MAE: 26.281667709350586, RMSE: 28.470375061035156


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 1456/1459 [03:30<00:00,  6.43it/s]

batch 1454, R-squared: -2.685825870495962, MAE: 18.918014526367188, RMSE: 21.55265235900879
batch 1455, R-squared: -3.0299337995122193, MAE: 24.6048641204834, RMSE: 27.066173553466797


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1459/1459 [03:31<00:00,  6.91it/s]

batch 1456, R-squared: -3.032946298172288, MAE: 26.825563430786133, RMSE: 29.599109649658203
batch 1457, R-squared: -2.7687453528845642, MAE: 23.903072357177734, RMSE: 27.509294509887695
batch 1458, R-squared: -3.8475882595581, MAE: 25.944541931152344, RMSE: 28.85982894897461
validation loss: 2.867105807892013, R-squared: -2.7379139028163135, MAE: 22.236873626708984, RMSE: 25.337202072143555
